# PyBN basics

PyBN is constructed with the idea of ease of use, thus it is built from several compatible modules. The central one is the network module which defines some of the most popular flavor of boolean networks. Let's start by running a simple Boolean Network. The first ingridient we require is a graph. Since Boolean Networks are intrinsecally related to graphs we decided to separate the core structure from the Boolean network from its functionally. The graph must be presented as a list of adjacencies.

In [1]:
from pybn.graphs import uniform_graph

average_connectivity = 2.4
nodes = 8
graph = uniform_graph(nodes, average_connectivity)
graph

[(0, 5),
 (1, 1),
 (2, 2),
 (2, 4),
 (2, 6),
 (3, 0),
 (3, 2),
 (3, 6),
 (4, 0),
 (4, 1),
 (4, 2),
 (4, 3),
 (4, 5),
 (5, 2),
 (5, 7),
 (6, 5),
 (6, 7),
 (7, 2),
 (7, 3),
 (7, 6)]

Given a graph presented in this format it is easy to define the boolean network. Lets run it for 10 steps and have its state printed.

In [2]:
from pybn.networks import BooleanNetwork

network = BooleanNetwork(nodes, graph)
steps = 10

In [3]:
# Set a random initial state.
network.set_initial_state()
print(network.state)
# Perform several steps.
for _ in range(steps):
    network.step()
    print(network.state)

[0 1 1 0 1 0 0 1]
[0 0 0 0 0 1 1 0]
[1 1 0 1 0 0 1 0]
[0 0 1 1 0 1 1 0]
[0 1 1 1 0 0 0 0]
[0 0 0 1 0 0 0 0]
[0 1 1 1 0 0 1 0]
[0 0 0 1 0 0 0 0]
[0 1 1 1 0 0 1 0]
[0 0 0 1 0 0 0 0]
[0 1 1 1 0 0 1 0]


Now, since running networks without getting some information out of them is boring. We introduce the observers. Observers are in charge of extracting information as efficiently as possible from the evolution of the Boolean Networks. Let's calculate the entropy of the states of the previous network from 128 steps. The first thing we have to do is define and attach the observer to the network.

In [4]:
from pybn.observers import EntropyObserver

observers = [EntropyObserver(nodes=nodes)] 
network.attach_observers(observers)

Then we just simply need to run the network. Notice that since we are just running the network once all the errors are 0.

In [5]:
steps = 128

# Set a random initial state.
network.set_initial_state(observe=True)

# Perform several steps.
for _ in range(steps):
    network.step(observe=True)

# Signal the observer that the run concluded. 
network.update_observers(end_of_run=True)

# Get observer's summary.    
network.observers_summary()

Network entropy:	0.420 ± 0.450

Network entropy (per node):
0.115 ± 0.000,	1.000 ± 0.000,	1.000 ± 0.000,	0.000 ± 0.000,	0.065 ± 0.000,	
0.115 ± 0.000,	1.000 ± 0.000,	0.065 ± 0.000,	

Network complexity:	0.163 ± 0.173

Network complexity (per node):
0.408 ± 0.000,	0.000 ± 0.000,	0.000 ± 0.000,	0.000 ± 0.000,	0.245 ± 0.000,	
0.408 ± 0.000,	0.000 ± 0.000,	0.245 ± 0.000,	




# Multiple executions 

When running a network multiple times a few optimizations can be made. This time, let's use a Fuzzy Network to illustrate this and include another observer.

In [8]:
from pybn.networks import FuzzyBooleanNetwork
from pybn.observers import TransitionsObserver

First we create the graph.

In [9]:
average_connectivity = 5.8
nodes = 20
graph = uniform_graph(nodes, average_connectivity)

Then the Fuzzy Network.

In [10]:
base = 5
fuzzy = FuzzyBooleanNetwork(nodes, base, graph)

And we attach some observers. Note that the observers must be grouped with a list. The first small optimization we can do is to let the observer know in advance how many values runs will be averaging over.

In [11]:
runs = 10
observers = [EntropyObserver(nodes=nodes, runs=runs, base=base),
             TransitionsObserver(nodes=nodes, runs=runs)] 
fuzzy.attach_observers(observers)

In case we want to "prewarm" (transcient) the network before measuring anything, we can just "turn off" the observers and then turning them on againg afterwards. 

In [12]:
steps = 128
transcient = 128

# Perform several runs.
for _ in range(runs):
    
    # Set a random initial state.
    fuzzy.set_initial_state(observe=False)
    
    # Perform several pre-warm and unobserved steps.
    for _ in range(transcient):
        fuzzy.step(observe=False)
    
    # Pass the last state to the observers.
    fuzzy.update_observers()
    
    # Perform several observebed steps.
    for _ in range(steps):
        fuzzy.step(observe=True)
        
    # Signal the observers that the run concluded.
    fuzzy.update_observers(end_of_run=True)
        
# Get observer's summary.
fuzzy.observers_summary()

Network entropy:	0.413 ± 0.028

Network entropy (per node):
0.430 ± 0.001,	0.427 ± 0.003,	0.425 ± 0.006,	0.422 ± 0.010,	0.420 ± 0.011,	
0.414 ± 0.011,	0.429 ± 0.002,	0.427 ± 0.003,	0.406 ± 0.012,	0.428 ± 0.002,	
0.426 ± 0.004,	0.424 ± 0.010,	0.336 ± 0.017,	0.428 ± 0.003,	0.394 ± 0.010,	
0.426 ± 0.003,	0.426 ± 0.006,	0.401 ± 0.008,	0.352 ± 0.038,	0.428 ± 0.003,	


Network complexity:	0.967 ± 0.027

Network complexity (per node):
0.980 ± 0.000,	0.979 ± 0.002,	0.977 ± 0.004,	0.975 ± 0.007,	0.974 ± 0.008,	
0.970 ± 0.008,	0.980 ± 0.001,	0.978 ± 0.002,	0.964 ± 0.010,	0.979 ± 0.001,	
0.978 ± 0.002,	0.977 ± 0.007,	0.891 ± 0.021,	0.979 ± 0.002,	0.955 ± 0.008,	
0.978 ± 0.001,	0.978 ± 0.004,	0.960 ± 0.006,	0.907 ± 0.048,	0.979 ± 0.002,	



Network transition entropy:	0.914 ± 0.076

Network transition entropy (per node):
0.945 ± 0.031,	0.933 ± 0.020,	0.952 ± 0.023,	0.894 ± 0.059,	0.941 ± 0.023,	
0.961 ± 0.036,	0.746 ± 0.069,	0.980 ± 0.009,	0.958 ± 0.031,	0.859 ± 0.033,	
0.920 ± 0.033,	0.943 ± 0.03

In [13]:
fuzzy.observers[0].file_summary()

[('entropy', '0.413412,0.027770,\n'), ('complexity', '0.966926,0.027052,\n')]